In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from glob import glob

In [2]:
files = glob("data/Train_Batch_*.csv")
files

['data\\Train_Batch_1.csv',
 'data\\Train_Batch_2.csv',
 'data\\Train_Batch_3.csv']

In [3]:
dfs = [pd.read_csv(file) for file in files]
print(dfs[0].shape)
print(dfs[1].shape)
print(dfs[2].shape)

(19440, 17)
(12956, 18)
(6466, 19)


There is a problem we can't concat our data because files have different columns we need to pre-process every single file alone then concat them!

In [4]:
df1 = dfs[0].copy()
df2 = dfs[1].copy()
df3 = dfs[2].copy()

In [5]:
print(df1.shape)
print(df1.info())
df1.head(2)

(19440, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19440 entries, 0 to 19439
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     19440 non-null  object 
 1   Person Description             19440 non-null  object 
 2   Place Code                     19440 non-null  object 
 3   Customer Order                 19440 non-null  object 
 4   Additional Features in market  15140 non-null  object 
 5   Promotion Name                 16530 non-null  object 
 6   Store Kind                     16530 non-null  object 
 7   Store Sales                    19440 non-null  object 
 8   Store Cost                     19440 non-null  object 
 9   Product Weights Data in (KG)   19440 non-null  object 
 10  Is Recyclable?                 19440 non-null  object 
 11  Min. Yearly Income             19437 non-null  object 
 12  Store Area                     174

,Unnamed: 0,Person Description,Place Code,Customer Order,Additional Features in market,Promotion Name,Store Kind,Store Sales,Store Cost,Product Weights Data in (KG),Is Recyclable?,Min. Yearly Income,Store Area,Grocery Area,Frozen Area,Meat Area,Cost
0,mc_ID_0,"Single Female with four children, education: b...",H11go_ZA,"Cleaning Supplies from Household department, O...","['Video Store', 'Florist', 'Ready Food', 'Coff...",Dimes Off,Deluxe,8.76 Millions,4.2924 Millions,"{'Gross Weight': 28.1997, 'Net Weight': 26.600...",recyclable,10K+,2842.23,2037.64,481.98,323,602.7575
1,mc_ID_1,"Single Female with three children, education: ...",S04ne_WA,"Snack Foods from Snack Foods department, Order...",NaN,Budget Bargains,Supermarket,6.36 Millions,1.9716 Millions,"{'Gross Weight': 16.571, 'Net Weight': 14.972,...",non recyclable,50K+,2814.95,2049.72,457.36,NaN,708.6650


change id table name and set index to id

In [6]:
def set_index(df):
    df = (
        df
        .rename(columns={"Unnamed: 0": "id"})
        .set_index("id")
    )
    return df

In [7]:
df1 = set_index(df1)
df1.head(2)

,Person Description,Place Code,Customer Order,Additional Features in market,Promotion Name,Store Kind,Store Sales,Store Cost,Product Weights Data in (KG),Is Recyclable?,Min. Yearly Income,Store Area,Grocery Area,Frozen Area,Meat Area,Cost
id,,,,,,,,,,,,,,,,
mc_ID_0,"Single Female with four children, education: b...",H11go_ZA,"Cleaning Supplies from Household department, O...","['Video Store', 'Florist', 'Ready Food', 'Coff...",Dimes Off,Deluxe,8.76 Millions,4.2924 Millions,"{'Gross Weight': 28.1997, 'Net Weight': 26.600...",recyclable,10K+,2842.23,2037.64,481.98,323,602.7575
mc_ID_1,"Single Female with three children, education: ...",S04ne_WA,"Snack Foods from Snack Foods department, Order...",NaN,Budget Bargains,Supermarket,6.36 Millions,1.9716 Millions,"{'Gross Weight': 16.571, 'Net Weight': 14.972,...",non recyclable,50K+,2814.95,2049.72,457.36,NaN,708.6650


In [8]:
def split_person_description(df):
    
    df[["personal", "deg_work"]] = (
        df["Person Description"]
        .str
        .split(", education: ", expand=True)
    )
    
    df[["Marriage", "Gender", "with", "Children", "tc"]] = (
        df["personal"]
        .str
        .split(expand=True)
    )
    
    df[["Degree", "Work"]] = (
        df["deg_work"]
        .str
        .split("working as", expand=True)
    )
    
    df = df.drop(columns=["Person Description", "personal", "with", "tc", "deg_work"])
    return df

In [9]:
df1 = split_person_description(df1)
df1.head(2)

,Place Code,Customer Order,Additional Features in market,Promotion Name,Store Kind,Store Sales,Store Cost,Product Weights Data in (KG),Is Recyclable?,Min. Yearly Income,Store Area,Grocery Area,Frozen Area,Meat Area,Cost,Marriage,Gender,Children,Degree,Work
id,,,,,,,,,,,,,,,,,,,,
mc_ID_0,H11go_ZA,"Cleaning Supplies from Household department, O...","['Video Store', 'Florist', 'Ready Food', 'Coff...",Dimes Off,Deluxe,8.76 Millions,4.2924 Millions,"{'Gross Weight': 28.1997, 'Net Weight': 26.600...",recyclable,10K+,2842.23,2037.64,481.98,323,602.7575,Single,Female,four,bachelors degree,professional
mc_ID_1,S04ne_WA,"Snack Foods from Snack Foods department, Order...",NaN,Budget Bargains,Supermarket,6.36 Millions,1.9716 Millions,"{'Gross Weight': 16.571, 'Net Weight': 14.972,...",non recyclable,50K+,2814.95,2049.72,457.36,NaN,708.6650,Single,Female,three,bachelors degree,management


In [10]:
df1["Place Code"].value_counts()

T02ma_WA    1802
S01em_OR    1771
P07nd_OR    1668
S03le_WA    1626
H11go_ZA    1508
M10da_YU    1452
S04ne_WA    1411
B06ls_CA    1355
L05es_CA    1322
B14on_WA    1109
V13er_BC    1073
O09ba_VE     861
C15ho_ZA     769
A17co_GU     503
M12ty_DF     429
S18co_CA     253
B08am_WA     228
V19ia_BC     177
G16ra_JA     123
Name: Place Code, dtype: int64

In [11]:
def split_customer_order(df):
    df[["ord_dep", "Oreder Brand"]] = (
        df["Customer Order"]
        .str
        .split(", Ordered Brand : ", expand=True)
    )
    
    df[["Product", "Department", "blank"]] = (
        df["ord_dep"]
        .str
        .split("from | department", expand=True)
    )
        
    df = df.drop(columns=["Customer Order", "ord_dep", "blank"])
    return df

In [12]:
df1 = split_customer_order(df1)
df1.head(3)

,Place Code,Additional Features in market,Promotion Name,Store Kind,Store Sales,Store Cost,Product Weights Data in (KG),Is Recyclable?,Min. Yearly Income,Store Area,...,Meat Area,Cost,Marriage,Gender,Children,Degree,Work,Oreder Brand,Product,Department
id,,,,,,,,,,,,,,,,,,,,,
mc_ID_0,H11go_ZA,"['Video Store', 'Florist', 'Ready Food', 'Coff...",Dimes Off,Deluxe,8.76 Millions,4.2924 Millions,"{'Gross Weight': 28.1997, 'Net Weight': 26.600...",recyclable,10K+,2842.23,...,323,602.7575,Single,Female,four,bachelors degree,professional,Red Wing,Cleaning Supplies,Household
mc_ID_1,S04ne_WA,NaN,Budget Bargains,Supermarket,6.36 Millions,1.9716 Millions,"{'Gross Weight': 16.571, 'Net Weight': 14.972,...",non recyclable,50K+,2814.95,...,NaN,708.6650,Single,Female,three,bachelors degree,management,Nationeel,Snack Foods,Snack Foods
mc_ID_2,L05es_CA,['Florist'],Shelf Emptiers,Supermarket,10.86 Millions,4.4526 Millions,"{'Gross Weight': 28.6358, 'Net Weight': 27.182...",recyclable,30K+,2192.32,...,348.85,564.2647,Married,Male,two,high school degree,skilled manual,Excel,Magazines,Periodicals


In [13]:
def encode_market_features(df):
    unique_feat = set()
    
    for feat_list in df["Additional Features in market"]:
        if pd.notna(feat_list):
            string_data = feat_list.strip("[]")
            elements = string_data.split(', ')
            elements = [element.strip("'") for element in elements]
            unique_feat.update(elements)

    for feat in unique_feat:
        df1[feat] = (
            df1["Additional Features in market"]
            .apply(lambda x: 1 if pd.notna(x) and feat in x else 0)
        )
        
    df = df.drop(columns="Additional Features in market")
    return df

In [14]:
df1 = encode_market_features(df1)
df1.head(3)

,Place Code,Promotion Name,Store Kind,Store Sales,Store Cost,Product Weights Data in (KG),Is Recyclable?,Min. Yearly Income,Store Area,Grocery Area,...,Degree,Work,Oreder Brand,Product,Department,Florist,Bar For Salad,Video Store,Ready Food,Coffee Bar
id,,,,,,,,,,,,,,,,,,,,,
mc_ID_0,H11go_ZA,Dimes Off,Deluxe,8.76 Millions,4.2924 Millions,"{'Gross Weight': 28.1997, 'Net Weight': 26.600...",recyclable,10K+,2842.23,2037.64,...,bachelors degree,professional,Red Wing,Cleaning Supplies,Household,1,1,1,1,1
mc_ID_1,S04ne_WA,Budget Bargains,Supermarket,6.36 Millions,1.9716 Millions,"{'Gross Weight': 16.571, 'Net Weight': 14.972,...",non recyclable,50K+,2814.95,2049.72,...,bachelors degree,management,Nationeel,Snack Foods,Snack Foods,0,0,0,0,0
mc_ID_2,L05es_CA,Shelf Emptiers,Supermarket,10.86 Millions,4.4526 Millions,"{'Gross Weight': 28.6358, 'Net Weight': 27.182...",recyclable,30K+,2192.32,1322.21,...,high school degree,skilled manual,Excel,Magazines,Periodicals,1,0,0,0,0


In [15]:
df1["Promotion Name"].value_counts()

Save Price                  676
Weekend Discount            660
Two Day Sale                633
Price Winners               579
Super Savers                568
Save It (Sale)              541
One Day Sale                519
Super Duper Savers          508
Roller Savings (High)       497
GLD                         478
Price Slashers              476
Full Free                   463
Shelf Clearing Days         459
Sale : Double Down          457
Lottery Cash Registerion    418
Two for One                 407
Big Time Discounts          400
Go For It                   394
Money Savers                379
Price Destroyers            369
Budget Bargains             360
Saving Days                 351
Discount Frenzy             332
Best Price Savers           327
Price Cutters               323
Dimes Off                   301
Savings Galore              298
Bag Stuffer                 287
Promo Big                   285
You Save Days               283
Sales Days                  279
I Cant B

In [16]:
df1["Store Kind"].value_counts()

Supermarket      7209
Deluxe           6220
Gourmet          1821
Mid-Size          761
Small Grocery     519
Name: Store Kind, dtype: int64

In [17]:
def transform_cost_sales(df):
    df["Store Sales"] = df1["Store Sales"].str.split(expand=True)[0].astype(float)
    df["Store Cost"] = df1["Store Cost"].str.split(expand=True)[0].astype(float)
    return df

In [18]:
df1 = transform_cost_sales(df1)
df1.head(3)

,Place Code,Promotion Name,Store Kind,Store Sales,Store Cost,Product Weights Data in (KG),Is Recyclable?,Min. Yearly Income,Store Area,Grocery Area,...,Degree,Work,Oreder Brand,Product,Department,Florist,Bar For Salad,Video Store,Ready Food,Coffee Bar
id,,,,,,,,,,,,,,,,,,,,,
mc_ID_0,H11go_ZA,Dimes Off,Deluxe,8.76,4.2924,"{'Gross Weight': 28.1997, 'Net Weight': 26.600...",recyclable,10K+,2842.23,2037.64,...,bachelors degree,professional,Red Wing,Cleaning Supplies,Household,1,1,1,1,1
mc_ID_1,S04ne_WA,Budget Bargains,Supermarket,6.36,1.9716,"{'Gross Weight': 16.571, 'Net Weight': 14.972,...",non recyclable,50K+,2814.95,2049.72,...,bachelors degree,management,Nationeel,Snack Foods,Snack Foods,0,0,0,0,0
mc_ID_2,L05es_CA,Shelf Emptiers,Supermarket,10.86,4.4526,"{'Gross Weight': 28.6358, 'Net Weight': 27.182...",recyclable,30K+,2192.32,1322.21,...,high school degree,skilled manual,Excel,Magazines,Periodicals,1,0,0,0,0


In [19]:
def extract_product_weights(df):
    
    df[["b1", "Gross Weight", "Net Weight", "Package Weight", "b2"]] = (
        df1["Product Weights Data in (KG)"]
        .str
        .split("{'Gross Weight': |, 'Net Weight': |, 'Package Weight': |}", expand=True)
    )
    
    df = df.drop(columns=["b1", "b2", "Product Weights Data in (KG)"])
    return df

In [20]:
df1 = extract_product_weights(df1)
df1.head(3)

,Place Code,Promotion Name,Store Kind,Store Sales,Store Cost,Is Recyclable?,Min. Yearly Income,Store Area,Grocery Area,Frozen Area,...,Product,Department,Florist,Bar For Salad,Video Store,Ready Food,Coffee Bar,Gross Weight,Net Weight,Package Weight
id,,,,,,,,,,,,,,,,,,,,,
mc_ID_0,H11go_ZA,Dimes Off,Deluxe,8.76,4.2924,recyclable,10K+,2842.23,2037.64,481.98,...,Cleaning Supplies,Household,1,1,1,1,1,28.1997,26.6008,1.599
mc_ID_1,S04ne_WA,Budget Bargains,Supermarket,6.36,1.9716,non recyclable,50K+,2814.95,2049.72,457.36,...,Snack Foods,Snack Foods,0,0,0,0,0,16.571,14.972,1.599
mc_ID_2,L05es_CA,Shelf Emptiers,Supermarket,10.86,4.4526,recyclable,30K+,2192.32,1322.21,523.32,...,Magazines,Periodicals,1,0,0,0,0,28.6358,27.1822,1.4536


In [21]:
def transform_recyclable(df):
    mapping = {'recyclable': 'yes', 'non recyclable': 'no'}
    df["Is Recyclable?"] = df["Is Recyclable?"].map(mapping)
    return df

In [22]:
df1 = transform_recyclable(df1)
df1.head(3)

,Place Code,Promotion Name,Store Kind,Store Sales,Store Cost,Is Recyclable?,Min. Yearly Income,Store Area,Grocery Area,Frozen Area,...,Product,Department,Florist,Bar For Salad,Video Store,Ready Food,Coffee Bar,Gross Weight,Net Weight,Package Weight
id,,,,,,,,,,,,,,,,,,,,,
mc_ID_0,H11go_ZA,Dimes Off,Deluxe,8.76,4.2924,yes,10K+,2842.23,2037.64,481.98,...,Cleaning Supplies,Household,1,1,1,1,1,28.1997,26.6008,1.599
mc_ID_1,S04ne_WA,Budget Bargains,Supermarket,6.36,1.9716,no,50K+,2814.95,2049.72,457.36,...,Snack Foods,Snack Foods,0,0,0,0,0,16.571,14.972,1.599
mc_ID_2,L05es_CA,Shelf Emptiers,Supermarket,10.86,4.4526,yes,30K+,2192.32,1322.21,523.32,...,Magazines,Periodicals,1,0,0,0,0,28.6358,27.1822,1.4536


In [23]:
def transform_income(df):
    df["Min. Yearly Income"] = (
        df1["Min. Yearly Income"]
        .str
        .split("K+", expand=True)[0]
        .astype(float) * 1000
    )
    return df

In [24]:
df1 = transform_income(df1)
df1.head()

,Place Code,Promotion Name,Store Kind,Store Sales,Store Cost,Is Recyclable?,Min. Yearly Income,Store Area,Grocery Area,Frozen Area,...,Product,Department,Florist,Bar For Salad,Video Store,Ready Food,Coffee Bar,Gross Weight,Net Weight,Package Weight
id,,,,,,,,,,,,,,,,,,,,,
mc_ID_0,H11go_ZA,Dimes Off,Deluxe,8.76,4.2924,yes,10000.0,2842.23,2037.64,481.98,...,Cleaning Supplies,Household,1,1,1,1,1,28.1997,26.6008,1.599
mc_ID_1,S04ne_WA,Budget Bargains,Supermarket,6.36,1.9716,no,50000.0,2814.95,2049.72,457.36,...,Snack Foods,Snack Foods,0,0,0,0,0,16.571,14.972,1.599
mc_ID_2,L05es_CA,Shelf Emptiers,Supermarket,10.86,4.4526,yes,30000.0,2192.32,1322.21,523.32,...,Magazines,Periodicals,1,0,0,0,0,28.6358,27.1822,1.4536
mc_ID_3,S03le_WA,Savings Galore,NaN,1.98,0.6732,yes,30000.0,1974.73,NaN,440.92,...,Dairy,Dairy,0,0,0,0,1,14.2161,11.2944,2.9217
mc_ID_4,M10da_YU,Sale Winners,Deluxe,11.56,4.9708,no,50000.0,2862.3,1872.19,593.93,...,Vegetables,Produce,1,1,1,1,1,12.6172,9.71,2.9072


In [25]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19440 entries, mc_ID_0 to mc_ID_19354
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Place Code          19440 non-null  object 
 1   Promotion Name      16530 non-null  object 
 2   Store Kind          16530 non-null  object 
 3   Store Sales         19440 non-null  float64
 4   Store Cost          19440 non-null  float64
 5   Is Recyclable?      19440 non-null  object 
 6   Min. Yearly Income  19437 non-null  float64
 7   Store Area          17483 non-null  object 
 8   Grocery Area        17460 non-null  object 
 9   Frozen Area         17506 non-null  float64
 10  Meat Area           17492 non-null  object 
 11  Cost                19416 non-null  float64
 12  Marriage            19440 non-null  object 
 13  Gender              19440 non-null  object 
 14  Children            19440 non-null  object 
 15  Degree              19440 non-null  object 
 1